In [1]:
# Import libraries
import pandas as pd
import json
from pymongo import MongoClient
import mysql.connector
import time
from sqlalchemy import create_engine,text
from datetime import date

In [2]:
df_answers = pd.read_csv("dataset/Answers.csv", encoding="ISO-8859-1")
df_questions = pd.read_csv("dataset/Questions.csv", encoding="ISO-8859-1")
df_taglink = pd.read_csv("dataset/Tags.csv", encoding="ISO-8859-1")
df_tags = df_taglink.drop_duplicates(subset=['Tag'], keep='first').drop(columns=['Id'])
df_tags.head()

,Tag
0,python
1,osx
2,fonts
3,photoshop
5,windows


In [3]:
# verificar as perguntas de 2015 com a palavra pdf
# df info de todos os datasets
print(f'df_questions')
print({df_questions.info()})
print("\n------------------------------------------\n")
print(f'df_answers')
print(f'{df_answers.info()}')
print("\n------------------------------------------\n")
print(f'df_tags')
print(f'{df_tags.info()}')
print("\n------------------------------------------\n")
print(f'df_taglink')
print(f'{df_taglink.info()}')

df_questions
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607282 entries, 0 to 607281
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            607282 non-null  int64  
 1   OwnerUserId   601070 non-null  float64
 2   CreationDate  607282 non-null  object 
 3   Score         607282 non-null  int64  
 4   Title         607282 non-null  object 
 5   Body          607282 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 27.8+ MB
{None}

------------------------------------------

df_answers
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987122 entries, 0 to 987121
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            987122 non-null  int64  
 1   OwnerUserId   981755 non-null  float64
 2   CreationDate  987122 non-null  object 
 3   ParentId      987122 non-null  int64  
 4   Score         987122

In [4]:
df_questions['CreationDate'] = pd.to_datetime(df_questions['CreationDate'])
df_answers['CreationDate'] = pd.to_datetime(df_answers['CreationDate'])

In [5]:
#limpeza
df_questions = df_questions.dropna()
df_questions['OwnerUserId'] = df_questions['OwnerUserId'].apply(int)
df_questions['Title'] = df_questions['Title'].astype('string')
df_questions['Body'] = df_questions['Body'].astype('string')

In [6]:
df_questions['year'] = df_questions['CreationDate'].dt.year
df_questions['month'] = df_questions['CreationDate'].dt.month
df_questions['day'] = df_questions['CreationDate'].dt.day
df_questions['hour'] = df_questions['CreationDate'].dt.hour
df_questions['minute'] = df_questions['CreationDate'].dt.minute
df_questions['second'] = df_questions['CreationDate'].dt.second

df_questions = df_questions.drop(columns=['CreationDate'])

df_questions['Date1'] = df_questions[df_questions.columns[5:8]].apply(
     lambda x: '/'.join(x.dropna().astype(str)),
     axis=1
)

df_questions['Date2'] = df_questions[df_questions.columns[8:11]].apply(
     lambda x: ':'.join(x.dropna().astype(str)),
     axis=1
)

df_questions['CreationDate'] = df_questions[df_questions.columns[11:13]].apply(
     lambda x: ' '.join(x.dropna().astype(str)),
     axis=1
)

df_questions = df_questions.drop(columns=['year', 'month', 'day', 'hour', 'minute', 'second', 'Date1', 'Date2'])

In [7]:
df_answers['year'] = df_answers['CreationDate'].dt.year
df_answers['month'] = df_answers['CreationDate'].dt.month
df_answers['day'] = df_answers['CreationDate'].dt.day
df_answers['hour'] = df_answers['CreationDate'].dt.hour
df_answers['minute'] = df_answers['CreationDate'].dt.minute
df_answers['second'] = df_answers['CreationDate'].dt.second

df_answers = df_answers.drop(columns=['CreationDate'])

df_answers['Date1'] = df_answers[df_answers.columns[5:8]].apply(
     lambda x: '/'.join(x.dropna().astype(str)),
     axis=1
)

df_answers['Date2'] = df_answers[df_answers.columns[8:11]].apply(
     lambda x: ':'.join(x.dropna().astype(str)),
     axis=1
)

df_answers['CreationDate'] = df_answers[df_answers.columns[11:13]].apply(
     lambda x: ' '.join(x.dropna().astype(str)),
     axis=1
)

df_answers = df_answers.drop(columns=['year', 'month', 'day', 'hour', 'minute', 'second', 'Date1', 'Date2'])

In [8]:
#convert CreationDate to datetime
df_answers['CreationDate'] = pd.to_datetime(df_answers['CreationDate'])
df_questions['CreationDate'] = pd.to_datetime(df_questions['CreationDate'])

In [9]:
df_answers.head()

,Id,OwnerUserId,ParentId,Score,Body,CreationDate
0,497,50.0,469,4,<p>open up a terminal (Applications-&gt;Utilit...,2008-08-02 16:56:53
1,518,153.0,469,2,<p>I haven't been able to find anything that d...,2008-08-02 17:42:28
2,536,161.0,502,9,<p>You can use ImageMagick's convert utility f...,2008-08-02 18:49:07
3,538,156.0,535,23,<p>One possibility is Hudson. It's written in...,2008-08-02 18:56:56
4,541,157.0,535,20,"<p>We run <a href=""http://buildbot.net/trac"">B...",2008-08-02 19:06:40


In [11]:
#create a new df_question dataframe with a column with a list of the ids of the awnsers correrpoding to the question by parentid
df_questions['answers'] = df_questions['Id'].apply(lambda x: list(df_answers[df_answers['ParentId'] == x]['Id']))
df_questions.head()


,Id,OwnerUserId,Score,Title,Body,CreationDate,answers
0,469,147,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,2008-08-02 15:11:16,"[497, 518, 3040, 195170]"
1,502,147,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,2008-08-02 17:01:58,"[536, 7073, 7090]"
2,535,154,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,2008-08-02 18:43:54,"[538, 541, 660, 61746, 74452, 117712, 9120453]"
3,594,116,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...,2008-08-03 01:15:08,"[595, 25213, 125140]"
4,683,199,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...,2008-08-03 13:19:16,"[701, 735, 745, 750, 31126, 31188, 57833, 4905..."


In [12]:
df_question_2015 = df_questions[df_questions['CreationDate'].dt.year == 2015]
df_question_2015_pdf = df_question_2015[df_question_2015['Title'].str.contains('pdf', case=False)]
df_question_2015_pdf_4 = df_question_2015_pdf[df_question_2015_pdf['answers'].apply(len) >= 4]
df_question_2015_pdf_4.head()



,Id,OwnerUserId,Score,Title,Body,CreationDate,answers
462988,33917637,5011347,0,Extract specific data from .pdf and save in Ex...,<p>Every month I need extract some data from ....,2015-11-25 13:23:15,"[33918209, 33918241, 33918354, 33921230]"
464850,33998802,5011347,1,Get list of pdf files in folder,<p>I want get a list of files name of all pdf ...,2015-11-30 12:34:03,"[33998874, 33998914, 33998943, 33998948]"


In [13]:
# Setting MongoDB database

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017")

# Drop and create database
client.drop_database("stackoverflow")
db = client['stackoverflow']

# Drop and create collections
db['Questions'].drop()
db['Answers'].drop()
db['Tags'].drop()
db['TagLink'].drop()
collection_questions = db['Questions']
collection_answers = db['Answers']
collection_tags = db['Tags']
collection_taglink = db['TagLink']

# Function to insert data into MongoDB
def insert_into_mongo(df, collection):
    initial_count = collection.count_documents({})
    print(f"# documents: {initial_count}") 
    try:
        data_dict = df.to_dict(orient="records")
        collection.insert_many(data_dict)
        valid_count = collection.count_documents({}) - initial_count
        print(f"{valid_count} documents inserted.")
    except Exception as e:
        attempted_count = len(data_dict)
        valid_count = collection.count_documents({}) - initial_count
        print(f"{valid_count} documents inserted")
        print(f"{attempted_count - valid_count} documents failed to insert")

insert_into_mongo(df_questions, collection_questions)
insert_into_mongo(df_answers, collection_answers)
insert_into_mongo(df_taglink, collection_taglink)
insert_into_mongo(df_tags, collection_tags)

# documents: 0
601070 documents inserted.
# documents: 0
987122 documents inserted.
# documents: 0
1885078 documents inserted.
# documents: 0
16896 documents inserted.


In [ ]:
#Connect to MySQL
username = 'root'
password = '!Leandro02#'
host = 'localhost' 
port = '3306' 
database_name = 'stackoverflow'
questions_table = 'questions'
answers_table = 'answers'
tags_table = 'tags'
taglink_table = 'taglink'

#Creating Table
# mycursor.execute("CREATE TABLE questions (id INT AUTO_INCREMENT PRIMARY KEY, OwnerUserId INT, CreationDate VARCHAR(255), score INT, title VARCHAR(255), body VARCHAR(255))")
# mycursor.execute("CREATE TABLE answers (id INT AUTO_INCREMENT PRIMARY KEY, OwnerUserId INT, CreationDate VARCHAR(255), parentid INT, score INT, body VARCHAR(255)) foreign key(parentid) references questions(id)")
# mycursor.execute("CREATE TABLE tags (tag VARCHAR(255) PRIMARY KEY)")
# mycursor.execute("CREATE TABLE taglink (id INT, tag VARCHAR(255)) foreign key(id) references questions(id) foreign key(tag) references tags(tag)")
# Check if Table Exists
# mycursor.execute("SHOW TABLES")
# for x in mycursor:
#     print(x)

# Create the connection URL for SQL Alchemy
db_url = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}'
engine = create_engine(db_url)
with engine.connect() as connection:
    connection.execute(text(f"DROP DATABASE IF EXISTS {database_name}"))
    connection.execute(text(f"CREATE DATABASE {database_name}"))
db_url = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(db_url)

df_questions.to_sql(questions_table, con=engine, if_exists='replace', index=False, method='multi')
df_answers.to_sql(answers_table, con=engine, if_exists='replace', index=False, method='multi')
df_tags.to_sql(tags_table, con=engine, if_exists='replace', index=False, method='multi')
df_taglink.to_sql(taglink_table, con=engine, if_exists='replace', index=False, method='multi')

In [ ]:
queries = [
    {"description": "Retrieve all questions that have a score greater than 300", "query": ""},
    
    {"description": "Retrieve all answers from 2008", "query": ""},
    
    {"description": "Insert new tag in first question from 2015 that has 2 tags", "query": ""},

    {"description": "Insert answer for the most recent question from 2015 about PDF that have more than 4 answers", "query": ""}

]

In [ ]:
df_tags.head()

,Tag
0,python
1,osx
2,fonts
3,photoshop
5,windows


In [31]:
from datetime import datetime
#find all answers from 2008 in mongodb using regex
query = {'CreationDate': {
                '$gte': datetime(2015, 1, 1),
                '$lt': datetime(2016, 1, 1)
            }, 
            'Title': {'$regex' : '.*pdf.*'}, 
            'answers': {'$size': 4}
            }

answers_2008 = collection_questions.find(query)
for answer in answers_2008:
    print(answer)


{'_id': ObjectId('6744650638feca7187f65415'), 'Id': 33917637, 'OwnerUserId': 5011347, 'Score': 0, 'Title': 'Extract specific data from .pdf and save in Excel file', 'Body': '<p>Every month I need extract some data from .pdf files to create an Excel table.</p>\n\n<p>I\'m able to convert the .pdf file to text but I\'m not sure how to extract and save the specific information I want. Now I have this code:</p>\n\n<pre><code>from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter\nfrom pdfminer.converter import TextConverter\nfrom pdfminer.layout import LAParams\nfrom pdfminer.pdfpage import PDFPage\nfrom cStringIO import StringIO\n\ndef convert_pdf_to_txt(path):\n    rsrcmgr = PDFResourceManager()\n    retstr = StringIO()\n    codec = \'utf-8\'\n    laparams = LAParams()\n    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)\n    fp = file(path, \'rb\')\n    interpreter = PDFPageInterpreter(rsrcmgr, device)\n    password = ""\n    maxpages = 0\n    cachi

In [32]:
mongo_times = []
for q in queries:
    start_time = time.time()
    
    if "score greater" in q["description"]:
        query = {'Score': {'$gt': 300}}
        result = collection_questions.find(query)

    elif "answers from 2008" in q["description"]:
        query = {
            'CreationDate': {
                '$gte': datetime(2008, 1, 1),
                '$lt': datetime(2009, 1, 1)
            }
        }
        result = collection_answers.find(query)

    # elif "2 tags" in q["description"]:
        # query = {}
        # result = 
    else:
        query = {'CreationDate': {
                '$gte': datetime(2015, 1, 1),
                '$lt': datetime(2016, 1, 1)
            }, 
            'Title': {'$regex' : '.*pdf.*'}, 
            'answers': {'$size': 4}}
        result = collection_questions.find(query).limit(1)
        
    end_time = time.time()
 
    list_result = list(result)
    mongo_times.append(end_time - start_time)
    print(f"{q['description']}: {len(list_result)} found in {end_time - start_time:.4f} seconds")

Retrieve all questions that have a score greater than 300: 241 found in 0.0000 seconds
Retrieve all answers from 2008: 7039 found in 0.0000 seconds
Retrive all questions that have 2 tags: 1 found in 0.0000 seconds
Retrive all questions from 2015 about PDF that have more than 4 answers: 1 found in 0.0000 seconds


In [28]:
#find the questions with 4 answers
query = {'CreationDate': {
                '$gte': datetime(2015, 1, 1),
                '$lt': datetime(2016, 1, 1)
            }, 
            'Title': {'$regex' : '.*PDF.*'},
    'answers': {'$size': 4}}

result = collection_questions.find(query)
for r in result:
    print(r)

In [ ]:
mysql_times = []
with engine.connect() as connection:
    for q in queries:
        start_time = time.time()
        if "score greater" in q["description"]:
            result = connection.execute(text("SELECT title FROM questions WHERE score > 300"))

        elif "answers from 2008" in q["description"]:
            result = connection.execute(text("SELECT * FROM answers WHERE creationdate LIKE '2008%'"))
        
        #elif "2 tags" in q["description"]:
            #result = connection.execute(text("INSER INTO (SELECT id FROM tags GROUP BY id HAVING COUNT(tag) = 2) AS t JOIN questions q ON t.id = q.id"))
            
        else:
            result = connection.execute(text("""
                INSERT INTO answers (OwnerUserId, CreationDate, ParentId, Score, Body)
                VALUES (1, NOW(), q.Id, 0, 'This is an answer')
                FROM questions q
                WHERE q.CreationDate LIKE '2015%' AND q.Title LIKE '%PDF%'
                AND (SELECT COUNT(*) FROM answers a WHERE a.ParentId = q.Id) > 4
                ORDER BY q.CreationDate DESC
                LIMIT 1
            """))
        results =  result.fetchall()
        end_time = time.time()
        mysql_times.append(end_time - start_time)
        print(f"{q['description']}: {len(results)} results found in {end_time - start_time:.4f} seconds")

In [ ]:
comparison_df = pd.DataFrame({
    "Query": [q["description"] for q in queries],
    "MongoDB_Time(s)": mongo_times,
    "MySQL_Time(s)": mysql_times
})

print(comparison_df)